## Formula 1 Dataset

Formula 1 is an international car racing competition. 
Drivers from around the world compete for a title, and 
the races are held in various countries, making it a 
global event.

Each driver is part of a team called a “Constructor,” 
typically sponsored by a specific car manufacturer.

## Questions:

**a) Which three constructors had the highest number of total points between 1981 and 2020?**  
- How many total points did each of them get?  
- How do the total number of points for each constructor compare to the average across constructors?

**b) Which three constructors had the highest number of total points between 2001 and 2020?**  
- How many total points did each of them get?  
- How do the total number of points for each constructor compare to the average across constructors?

**c) How did the rankings change across the two time periods?**

**d) How many different drivers did Ferrari have between 1981 and 2020?**

**e) What was the best year for Ferrari between 1981 and 2020?**

In [3]:
import numpy as np
import pandas as pd

**a) Which three constructors had the highest number of total points between 1981 and 2020?**  
- How many total points did each of them get?  
- How do the total number of points for each constructor compare to the average across constructors?

#Import possible tables

In [4]:
constructor_results = pd.read_csv("/Users/jiangshuyue/Desktop/QTM/qtm530fall2024/Formula1/constructor_results.csv")
constructor_results

,constructorResultsId,raceId,constructorId,points,status
0,1,18,1,14.0,\N
1,2,18,2,8.0,\N
2,3,18,3,9.0,\N
3,4,18,4,5.0,\N
4,5,18,5,2.0,\N
...,...,...,...,...,...
12500,17005,1132,117,10.0,\N
12501,17006,1132,3,2.0,\N
12502,17007,1132,215,1.0,\N
12503,17008,1132,15,0.0,\N


In [5]:
constructor_information = pd.read_csv("/Users/jiangshuyue/Desktop/QTM/qtm530fall2024/Formula1/constructors.csv")
constructor_information

,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formul...
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso
...,...,...,...,...,...
207,210,haas,Haas F1 Team,American,http://en.wikipedia.org/wiki/Haas_F1_Team
208,211,racing_point,Racing Point,British,http://en.wikipedia.org/wiki/Racing_Point_F1_Team
209,213,alphatauri,AlphaTauri,Italian,http://en.wikipedia.org/wiki/Scuderia_AlphaTauri
210,214,alpine,Alpine F1 Team,French,http://en.wikipedia.org/wiki/Alpine_F1_Team


In [6]:
race_information = pd.read_csv("/Users/jiangshuyue/Desktop/QTM/qtm530fall2024/Formula1/races.csv")
race_information

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120,1140,2024,20,32,Mexico City Grand Prix,2024-10-27,20:00:00,https://en.wikipedia.org/wiki/2024_Mexico_City...,2024-10-25,18:30:00,2024-10-25,22:00:00,2024-10-26,17:30:00,2024-10-26,21:00:00,\N,\N
1121,1141,2024,21,18,São Paulo Grand Prix,2024-11-03,17:00:00,https://en.wikipedia.org/wiki/2024_S%C3%A3o_Pa...,2024-11-01,14:30:00,2024-11-01,18:30:00,\N,\N,2024-11-02,18:00:00,2024-11-02,14:00:00
1122,1142,2024,22,80,Las Vegas Grand Prix,2024-11-23,06:00:00,https://en.wikipedia.org/wiki/2024_Las_Vegas_G...,2024-11-21,02:30:00,2024-11-21,06:00:00,2024-11-22,02:30:00,2024-11-22,06:00:00,\N,\N
1123,1143,2024,23,78,Qatar Grand Prix,2024-12-01,17:00:00,https://en.wikipedia.org/wiki/2024_Qatar_Grand...,2024-11-29,13:30:00,2024-11-29,17:30:00,\N,\N,2024-11-30,17:00:00,2024-11-30,13:00:00


#Find race from 1981-2020

In [7]:
race_information_from_1981_to_2020 = race_information.query("(year >= 1981) and (year <= 2020)")
race_information_from_1981_to_2020

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1030,1043,2020,13,21,Emilia Romagna Grand Prix,2020-11-01,12:10:00,http://en.wikipedia.org/wiki/2020_Emilia_Romag...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1031,1044,2020,14,5,Turkish Grand Prix,2020-11-15,10:10:00,http://en.wikipedia.org/wiki/2020_Turkish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1032,1045,2020,15,3,Bahrain Grand Prix,2020-11-29,14:10:00,http://en.wikipedia.org/wiki/2020_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1033,1046,2020,16,3,Sakhir Grand Prix,2020-12-06,17:10:00,http://en.wikipedia.org/wiki/2020_Sakhir_Grand...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N


In [10]:
constructor_results_from_1981_to_2020 = pd.merge(constructor_results,
                                                 race_information_from_1981_to_2020,
                                                 on = "raceId",
                                                 how = "right")#save all the race from 1981 to 2020
constructor_results_from_1981_to_2020

,constructorResultsId,raceId,constructorId,points,status,year,round,circuitId,name,date,...,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,3823,1,23,18.0,\N,2009,1,1,Australian Grand Prix,2009-03-29,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1,3824,1,1,0.0,\N,2009,1,1,Australian Grand Prix,2009-03-29,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,3825,1,7,11.0,\N,2009,1,1,Australian Grand Prix,2009-03-29,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,3826,1,4,4.0,\N,2009,1,1,Australian Grand Prix,2009-03-29,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,3827,1,3,3.0,\N,2009,1,1,Australian Grand Prix,2009-03-29,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8573,16225,1047,211,1.0,\N,2020,17,24,Abu Dhabi Grand Prix,2020-12-13,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
8574,16226,1047,51,0.0,\N,2020,17,24,Abu Dhabi Grand Prix,2020-12-13,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
8575,16227,1047,6,0.0,\N,2020,17,24,Abu Dhabi Grand Prix,2020-12-13,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
8576,16228,1047,3,0.0,\N,2020,17,24,Abu Dhabi Grand Prix,2020-12-13,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N


In [ ]:
constructor_total_points = (constructor_results_from_1981_to_2020
                             .groupby(["constructorId"])
                             .agg(avg_points = ('points','mean'))
                             .sort_values(by = 'avg_points', ascending = False)
constructor_total_points